### Digit Recognizer

* Interactive Intro to Dimensionality Reduction(차원감소??)
 - https://www.kaggle.com/arthurtok/interactive-intro-to-dimensionality-reduction

### Introduction
특히 PCA(주 구성 요소 분석), LDA(선형 분류 분석), TSNE(T-분산 확률 이웃 포함)와 같은 Big 3의 차원 감소 방법의 장점을 다루는 많은 노트북이 이미 존재한다. 이것들 중 꽤 많은 부분이 서로 비교했지만, 한번에 3가지 모두를 모은 것은 거의 없다. 따라서, 이 노트북은 이 3가지 방법에 대해서 직관적으로 표현하기위해 그리는 것과 같은 기본적인 설명 제공과 보다 직관적으로 묘사하는 것을 목적으로 한다. 이 챕터는 다음과 같이 구성되어 있다.

1. Principal Component Analysis(PCA, 주 구성 요소 분석) - Unsupervised, linear method
2. Linear Discriminant Analysis(LDA, 선현 분류 분석) - Supervised, linear method
3. t-distributed Stochastic Neighbour Embedding(t-SNE) - Nonlinear, probabilistic method

* 신경망 (Neural Network) 에서 학습기가 분류 (classification) 하려는 대상이 자갈과 모래라는 것을 미리 알고서 훈련예 (training example) 로서 학습시켜 어떤 대상이 자갈에 속하는지 모래에 속하는지를 분류하는 것이 지도학습 (supervised learning)이다. 반면에 분류하려는 대상에 대한 어떤 정보도 주어지지 않고 학습기로 하여금 그것이 자갈인지 또는 모래인지 또는 그 밖의 어떤 것인지를 분류하는 것이 자율학습 (unsupervised learning)
* t-SNE : 데이터의 차원 축소에 사용되는 기계 학습 알고리즘. 고차원 데이터를 특히 2, 3차원 등으로 줄여 가시화하는데에 유용하게 사용. 비슷한 데이터는 선택될 확률이 매우 높지만 다른 데이터끼리는 선택될 확률이 매우 낮도록 설계된다.(http://bit.ly/2eQFos4)



In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
import seaborn as sns
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline

# Import the 3 dimensionality reduction methods
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA